In [1]:
#필요한 모듈 import, 설치

import numpy as np
import pandas as pd

!pip install keras
!pip install -q konlpy
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

#예측에서 사용
#! pip install krwordrank
#from krwordrank.word import KRWordRank

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 83.3 MB/s 
     |████████████████████████████████| 86 kB 4.8 MB/s 
Installing automake (A dependency for mecab-ko)
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 ht

In [2]:
#전처리한 파일을 사용

dfp = pd.read_csv('/content/drive/MyDrive/skku/인공지능앱개발대회/notice_preprocess.csv')

In [3]:
from konlpy.tag import Mecab

mecab = Mecab()

def get_tokens(x):
    try:
        return [i for i in mecab.nouns(x) if len(i) > 1] if x else []
    except Exception as e:
        if str(x) == 'nan':
            return []
        print(e)
        print(str(x))
        raise e

In [4]:
#mecab 계산해서 dfp에 저장

dfp['mecab'] = dfp['body'].map(get_tokens)
dfp['mecab_len'] = dfp['mecab'].map(len)
dfp = dfp[dfp['mecab_len'] >= 5]

In [5]:
from gensim.models import Word2Vec

model = Word2Vec(dfp['mecab'], size=100, window = 2, min_count=50, workers=4, iter=100, sg=1)

In [6]:
model

In [7]:
def get_sentence_mean_vector(morphs):
    vector = []
    for i in morphs:
        try:
            vector.append(model.wv[i])
        except KeyError as e:
            pass
    try:
        return np.mean(vector, axis=0)
    except IndexError as e:
        pass

In [8]:
tmp = dfp['mecab'].map(get_sentence_mean_vector).tolist()
tmp

[array([-0.04238267, -0.06660271,  0.07701457, -0.00186598, -0.31456435,
        -0.04521289, -0.22283505,  0.11532223,  0.06056834,  0.05670457,
        -0.0893427 ,  0.3587188 ,  0.09457372, -0.24180149, -0.21401204,
         0.23532416, -0.16522342,  0.04813299, -0.14072925,  0.01300786,
        -0.32993048, -0.09276231, -0.13001788, -0.11039183, -0.01435736,
         0.00595097,  0.36850452,  0.34336093,  0.02957452,  0.02850775,
         0.21678886, -0.03925656,  0.08031367,  0.2004577 ,  0.32659787,
        -0.09500754, -0.06652939,  0.12481976,  0.01803395,  0.13666432,
        -0.0406441 ,  0.12410121, -0.18500653,  0.03146652,  0.11435268,
        -0.1457767 ,  0.07013028,  0.20769408, -0.22592515,  0.04167775,
        -0.04066398,  0.06430443,  0.20512538, -0.02344123,  0.05600138,
        -0.10590443, -0.27506763, -0.01014373, -0.14340226,  0.34204686,
        -0.185966  ,  0.15818654, -0.1035178 ,  0.2122923 , -0.35168228,
         0.03266709,  0.13321097, -0.1747481 , -0.1

In [9]:
dfp['w2v'] = dfp['mecab'].map(get_sentence_mean_vector)

In [10]:
from sklearn.cluster import KMeans
import time

word_vectors = dfp.w2v.to_list() 
num_clusters = 25

# K-means
kmeans_clustering = KMeans(n_clusters = num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)
dfp['category'] = idx

In [11]:
dfp.head()

,title,text,body,mecab,mecab_len,w2v,category
0,프로지니어㈜ 연구원 채용,◎ 모집분야: \r\n1. 신약개발 연구원\r\n담당업무:\r\n■ 분자생물학 관련...,모집분야 1 신약개발 연구원 담당업무 분자생물학 관련 실험 업무 혈액 조직에서 추출...,"[모집, 분야, 신약, 개발, 연구원, 담당, 업무, 분자, 생물학, 관련, 실험,...",221,"[-0.042382672, -0.066602714, 0.07701457, -0.00...",6
1,"독일프리드리히나우만재단, 미래인문학기반사회혁신창업교육연구단 공동 주최 “시민참여와 ...","독일프리드리히나우만재단, 미래인문학기반사회혁신창업교육연구단 공동 주최 \r\n\r...",독일프리드리히나우만재단 미래인문학기반사회혁신창업교육연구단 공동 주최 시민참여와 사회...,"[독일, 프리드리히, 나우만, 재단, 미래, 인문학, 기반, 사회, 혁신, 창업, ...",131,"[-0.03043705, 0.061178703, -0.04974598, -0.275...",14
2,[응용통계연구소] 2021년 하계 통계특강 개최(온라인 진행),<접수기간 및 접수방법>\r\n\r\n\r\n- R 기초 강좌 수강료 (등록비 1만...,접수기간 및 접수방법 기초 강좌 수강료 등록비 1만원 및 교재포함 강좌별별도 학생 ...,"[접수, 기간, 접수, 방법, 기초, 강좌, 수강료, 등록비, 교재, 포함, 강좌,...",100,"[0.29845807, -0.2463903, 0.06638148, 0.0026813...",24
3,[자연과학캠퍼스] 우편취급국(수원) 근로학생(졸업생 포함) 모집 (기간연장),성균관대학교 자연과학캠퍼스 우편취급국(수원 소재)에서 사무보조 업무를 수행할 근로학...,성균관대학교 자연과학캠퍼스 우편취급국 수원 소재 에서 사무보조 업무를 수행할 근로학...,"[성균관, 대학교, 자연, 과학, 캠퍼스, 우편, 취급, 수원, 소재, 사무, 보조...",112,"[0.040397342, -0.18950437, 0.16274202, 0.06802...",3
4,SW융합대학 글로벌융합학부 자체직원 모집 공고,우리 성균관대학교 SW융합대학 글로벌융합학부에서 아래와 같이 자체직원을 채용하오니 ...,우리 성균관대학교 융합대학 글로벌융합학부에서 아래와 같이 자체직원을 채용하오니 많은...,"[우리, 성균관, 대학교, 융합, 대학, 글로벌, 융합, 학부, 아래, 자체, 직원...",203,"[0.017651904, 0.012614534, 0.17193715, -0.0059...",3


In [12]:
dfp.category.value_counts()

21    1192
13    1153
3     1007
8      884
1      870
14     800
2      765
16     744
12     696
18     625
9      589
0      441
6      439
15     427
7      425
23     410
20     375
5      236
24     187
17     157
4      126
22      87
10      65
19      51
11      42
Name: category, dtype: int64

In [13]:
import pickle
from sklearn.externals import joblib

#만든 lda모델 저장

saved_model = pickle.dumps(kmeans_clustering)
#kmeans_clustering.save('kmeans.model')
#pickle.dump(kmeans, open('kmeans.pk', 'wb'))
# then reload it with
#kmeans_model = pickle.load('kmeans_model.pk')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [14]:
joblib.dump(kmeans_clustering, 'kmeans.pkl') 

['kmeans.pkl']

In [45]:
joblib.dump(model, 'w2v.pkl')

['w2v.pkl']

In [ ]:
! pip install krwordrank
from krwordrank.word import KRWordRank

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python-dev #파이썬 버전 맞춰서 
pip install JPype1
pip install konlpy

%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"


In [25]:
from konlpy.tag import Okt
tokenizer = Okt()

In [17]:
dfp = dfp.dropna()

In [18]:
wordrank_extractor = KRWordRank(
    min_count = 5,
    max_length = 10, 
    verbose = True
    )

beta = 0.85  
max_iter = 10

In [41]:
not_check = ['성균관대', '캠퍼스', '홈페이지', '확인', '경우', '위', '층', '제', '차']

In [ ]:
word_list = []
for i in range(num_clusters):
    tmp = []
    sentence = ""
    try:
        keywords, rank, graph = wordrank_extractor.extract(dfp[dfp['category'] == i].body.tolist(), beta, max_iter)
    except AttributeError:
        keywords, rank, graph = wordrank_extractor.extract(dfp[dfp['category'] == i].body.tolist(), beta, max_iter)
        for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:50]:
            sentence += word
        pos = tokenizer.pos(sentence, stem=True)
        for p in pos:
            if p[1] == 'Noun' and p[0] not in not_check: #p[1] == 'Verb' or p[1] == 'Josa' or p[1] == 'Number'
                tmp.append(p[0])
                if len(tmp) >= 30:
                    break
            '''
            try:
                word = int(word)
            except ValueError:
                tmp.append(word)
            '''
    word_list.append(tmp)

In [43]:
for words in word_list:
    print(words)

['신청', '사용', '학생', '촬영', '인사', '캠', '대여', '예약', '운동장', '전문의', '가능', '예비군', '안내', '시간', '아래', '단체', '학교', '학위', '복', '진행', '학기', '졸업', '해당', '학기', '장소교', '훈련', '변경', '접수', '반드시', '대상']
['지원', '활동', '신청', '대학', '모집', '학생', '참여', '제출', '봉사', '선발', '교육', '문의', '접수', '프로그램', '가능', '재학생', '한국', '멘토', '작성', '안내', '진행', '지급', '서울', '이상', '서류', '사업', '장학금', '통해', '운영', '참가']
['신청', '수강', '학생', '학기', '학기', '마감', '가능', '제출', '학점', '인정이', '수학과', '안내', '대학', '수업', '원문', '과목', '반드시', '이상', '성적', '해당', '학기', '학사', '기간', '교과목', '후예', '정확', '불가', '본교', '졸업', '선발']
['학교', '근무', '제출', '지원', '행정', '본교', '서류', '문의', '모집', '면접', '접수', '채용', '업무', '개별', '가능', '부대', '학우', '아래', '다음', '반환', '합격자', '수원', '기재', '학사', '명', '이메일', '이력서', '교육', '연구', '기타']
['세미나', '모시', '참석', '균', '나노', '과학기술원', '관심', '개최', '오니', '분', '교수', '에너지', '과학', '삼성', '장소', '수석', '박사', '포항', '일시', '고려', '참여', '서울대학교', '한국', '연구']
['신청', '학자금', '대출', '학생', '대학생', '등록금', '학기', '한국', '장학금', '학기', '환지', '이상', '제외', '이후', '소득', '반드시', '가능', '재학'

In [44]:
import csv

f = open('category_words.csv','w', newline='')
wr = csv.writer(f)

for words in word_list:
    wr.writerow(words)
 
f.close()

### 여기서부터는 해야하는 부분 모델, csv리스트 load한거 사용

In [47]:
kmeans = joblib.load('kmeans.pkl')
word2vec = joblib.load('w2v.pkl')

In [50]:
f = open('category_words.csv', 'r')

category_words = []
reader = csv.reader(f)
 
for line in reader:
    category_words.append(line)
 
f.close()

In [48]:
keyword = ['인공지능', '장학금', '채용', '근로', '취업', '경영학']

In [51]:
max_score = []
total = []
for k in keyword: 
    score = [0 for _ in range(len(category_words))]
    for i, words in enumerate(category_words):
        for w in words:
            try:
                sim = word2vec.wv.similarity(k, w)
            except KeyError:
                pass
            score[i] += sim

    print(k, max(score), score)
    max_score.append(max(score))
    total.append(score)

인공지능 5.951141394674778 [2.6268839600961655, 2.5039257863536477, 2.7049521221779287, 3.299176742322743, 4.920408323407173, 2.9665599898435175, 3.896624389104545, 4.467936523258686, 1.9782149377278984, 4.100265692919493, 4.82367192581296, 5.020658202469349, 4.650669569149613, 2.8939134078100324, 5.8744461834430695, 5.951141394674778, 1.950310075422749, 3.3936549918726087, 2.7643424374982715, 4.756532285362482, 3.4254686336498708, 4.123992887092754, 5.6307180523872375, 5.020335872657597, 2.781505495076999]
장학금 9.609931781888008 [4.573184390552342, 7.007408309727907, 5.50961535051465, 4.018448635935783, 2.322463257238269, 9.117066875100136, 3.7793446115683764, 3.4367029750719666, 9.609931781888008, 3.172753168269992, 3.8400859367102385, 2.8989364923909307, 3.60346763394773, 5.760737240314484, 2.9628494642674923, 3.1025363616645336, 4.026190826669335, 3.6085339905694127, 4.065221490338445, 1.900062182918191, 3.8588420255109668, 3.48338552005589, 1.7452598633244634, 4.302007429185323, 5.4803

In [63]:
index = 5

In [64]:
sim_idx = total[index].index(max_score[index]) #인공지능3 장학금8 채용3 근로16 취업11 경영학12
dfp[dfp['category'] == sim_idx].head(20)

,title,text,body,mecab,mecab_len,w2v,category
64,글로벌 코스메틱 차세대 리더 과정 1기 모집 안내(2021년 9월 개강),성균관대학교 화학공학과와 인재교육원에서 글로벌 코스메틱 차세대 리더 과정 1기를 다...,성균관대학교 화학공학과와 인재교육원에서 글로벌 코스메틱 차세대 리더 과정 1기를 다...,"[성균관, 대학교, 화학, 공학, 인재, 교육, 글로벌, 스메, 차세대, 리더, 과...",141,"[0.15145127, -0.022378776, 0.03966375, -0.0552...",12
253,"제1회 대학원 학생성공스토리 특강 개최 안내(5/27, 목 15시/해외포닥)","제1회 대학원 학생성공스토리 특강 \r\n""대학원생의 도전: 해외 포스트 닥터의 길...",제1회 대학원 학생성공스토리 특강 대학원생의 도전 해외 포스트 닥터의 길 국내 대학...,"[대학원, 학생, 성공, 스토리, 특강, 대학원생, 도전, 해외, 포스트, 닥터, ...",120,"[0.046118986, -0.02803058, -0.009154312, -0.13...",12
349,제59주기 심산 김창숙 선생 추모식 안내,심산 김창숙 선생 제59주기 추모식\r\n\r\n\r\n일시 : 2021.05.07...,심산 김창숙 선생 제59주기 추모식 일시 2021 05 07 금 11시 장소 심산열...,"[심산, 김창숙, 선생, 주기, 추모식, 일시, 장소, 심산, 사상, 인사, 심산,...",39,"[0.13894072, -0.2194526, 0.0036068328, -0.0818...",12
380,제59주기 심산 김창숙 선생 추모식 안내,심산 김창숙 선생 제59주기 추모식\r\n\r\n\r\n\r\n일시 : 2021.0...,심산 김창숙 선생 제59주기 추모식 일시 2021 05 07 금 11시 장소 심산열...,"[심산, 김창숙, 선생, 주기, 추모식, 일시, 장소, 심산, 사상, 인사, 심산,...",39,"[0.13894072, -0.2194526, 0.0036068328, -0.0818...",12
387,[성균갤러리] '해체구성 Deconstruction'전시 영상 안내,안녕하세요. 성균갤러리 전시 소식을 전합니다. \r\n\r\n'해체구성 Decons...,안녕하세요 성균갤러리 전시 소식을 전합니다 해체구성 전시 영상을 를 통해 공유합니다...,"[안녕, 갤러리, 전시, 소식, 해체, 구성, 전시, 영상, 공유, 링크, 해체, ...",18,"[0.3128951, -0.06455737, -0.09144775, -0.16559...",12
480,2021 글로벌융복합콘텐츠연구소 Virtual 국제학술대회 행사 안내,성균관대학교 글로벌융복합콘텐츠연구소(Global Convergence Content...,성균관대학교 글로벌융복합콘텐츠연구소 와 문화예술미디어융합원은 를 주제로 국제학술대회...,"[성균관, 대학교, 글로벌, 복합, 콘텐츠, 연구소, 문화, 예술, 미디어, 융합,...",102,"[0.08502304, -0.017142858, -0.09003307, -0.073...",12
510,2021학년도 후기 데이터사이언스 융합학과 입시설명회 개최 안내(4/9 금),안녕하십니까. 소프트웨어융합대학 데이터사이언스융합학과입니다.\r\n\r\n\r\n2...,안녕하십니까 소프트웨어융합대학 데이터사이언스융합학과입니다 2021학년도 1학기 대학...,"[안녕, 소프트웨어, 융합, 대학, 데이터, 사이언스, 융합, 학과, 학년도, 학기...",49,"[0.06432791, -0.0679246, -0.066847466, -0.0535...",12
538,2021학년도 후기 핀테크융합전공 석/박사과정 온라인 입학설명회 개최,"안녕하십니까, 핀테크융합전공 행정실입니다.\r\n\r\n\r\n\r\n핀테크융합전공...",안녕하십니까 핀테크융합전공 행정실입니다 핀테크융합전공에서 2021학년도 후기 온라인...,"[안녕, 핀테크, 융합, 전공, 행정, 핀테크, 융합, 전공, 학년도, 후기, 온라...",42,"[0.09606607, 0.07701038, -0.1921004, -0.074051...",12
583,[초청강연] 실리콘밸리 IT 전문가와 킹고인의 열네번째 만남: 실리콘밸리 청년들의 ...,안녕하세요 인터랙션사이언스학과입니다.\r\n아래와 같이 실리콘밸리 IT전문가를 초청...,안녕하세요 인터랙션사이언스학과입니다 아래와 같이 실리콘밸리 전문가를 초청한 열네번째...,"[안녕, 인터랙션, 사이언스, 학과, 아래, 실리콘밸리, 전문가, 초청, 열네, 강...",52,"[0.1360592, -0.03400189, -0.14039016, -0.21345...",12
584,[초청강연] 실리콘밸리 IT 전문가와 킹고인의 열다섯번째 만남: 데이터 분석의 ca...,안녕하세요 인터랙션사이언스학과입니다.\r\n아래와 같이 실리콘밸리 IT전문가를 초청...,안녕하세요 인터랙션사이언스학과입니다 아래와 같이 실리콘밸리 전문가를 초청한 열다섯번...,"[안녕, 인터랙션, 사이언스, 학과, 아래, 실리콘밸리, 전문가, 초청, 다섯, 강...",45,"[0.11152457, -0.018390235, -0.16133767, -0.204...",12
